In [1]:
import os
import urllib.request
import sys

def setup_windows_hadoop():
    # 1. Define Paths
    # We use C:/hadoop because it avoids space-in-path issues (like "Program Files")
    base_dir = "C:\\hadoop"
    bin_dir = os.path.join(base_dir, "bin")

    # 2. Create Directories
    print(f"Creating directory structure: {bin_dir}...")
    try:
        os.makedirs(bin_dir, exist_ok=True)
        print("   Directory created (or already exists).")
    except PermissionError:
        print(" ERROR: Permission denied. Try running PyCharm/Jupyter as Administrator.")
        return False

    # 3. Define Files to Download (Standard Hadoop 3.2.2 binaries for Spark 3.x)
    # Source: cdarlint/winutils (The standard community source)
    base_url = "https://github.com/cdarlint/winutils/raw/master/hadoop-3.2.2/bin/"
    files = ["winutils.exe", "hadoop.dll"]

    # 4. Download Files
    print("\n Downloading required binaries...")
    for filename in files:
        target_path = os.path.join(bin_dir, filename)
        url = base_url + filename

        if os.path.exists(target_path):
            print(f"    {filename} already exists. Skipping.")
        else:
            print(f"    Downloading {filename}...")
            try:
                urllib.request.urlretrieve(url, target_path)
                print(f"    Downloaded {filename}")
            except Exception as e:
                print(f"    Failed to download {filename}: {e}")
                return False

    # 5. Set Environment Variables for THIS session
    print("\n🔧 Configuring Environment Variables...")
    os.environ['HADOOP_HOME'] = base_dir
    # Add bin to PATH if not there
    if bin_dir not in os.environ['PATH']:
        os.environ['PATH'] = bin_dir + ";" + os.environ['PATH']

    print(" SUCCESS: Hadoop environment is set up.")
    print(f"   HADOOP_HOME = {os.environ['HADOOP_HOME']}")
    return True

# Execute
if sys.platform.startswith('win'):
    setup_windows_hadoop()
else:
    print("Skipping Windows setup (Not running on Windows).")

Skipping Windows setup (Not running on Windows).


In [2]:
import sys
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf

# ==========================================
# 1. PRODUCTION CONFIG
# ==========================================
IS_LOCAL = False          
USE_FULL_DATASET = True   

# --- GOOGLE CLOUD CONFIG ---
GCP_PROJECT_ID = "de2025-472211"
BQ_DATASET_NAME = "vehicles_data"
BQ_TABLE_NAME = "market_intelligence_system"
GCS_TEMP_BUCKET = "vehicles_csv"  # Bucket Name

# ==========================================
# 2. ENVIRONMENT CONFIGURATION
# ==========================================
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if IS_LOCAL:
    pass
else:
    print("☁️ Configuring for CLOUD/VM Environment...")
    os.environ['PYSPARK_PYTHON'] = sys.executable
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    
    # --- FIX: REMOVED '/input' FOLDER FROM PATHS ---
    # Based on your gsutil ls output: gs://vehicles_csv/vehicles.csv
    PATH_SMALL = f"gs://{GCS_TEMP_BUCKET}/first100_vehicles.csv" 
    PATH_FULL  = f"gs://{GCS_TEMP_BUCKET}/vehicles.csv"

INPUT_FILE = PATH_FULL if USE_FULL_DATASET else PATH_SMALL

# ==========================================
# 3. INITIALIZE SPARK
# ==========================================
conf = SparkConf()
conf.setAppName("Advanced_Market_Pipeline")
conf.set("spark.driver.memory", "4g")

# --- DEPENDENCIES ---
# 1. BigQuery Connector (Maven)
conf.set("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.36.1")

# 2. GCS Connector (Local Shaded JAR)
# Using the file you downloaded manually to /home/vanac
conf.set("spark.jars", "/home/vanac/gcs-connector-hadoop3-2.2.22-shaded.jar")

# Configure Hadoop
conf.set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
conf.set("spark.hadoop.google.cloud.auth.service.account.enable", "true")

print(f"🚀 Starting Spark Engine...")

try:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    spark.sparkContext.setLogLevel("WARN")
    print(f" SUCCESS: Spark {spark.version} is running.")
    print(f" Target File: {INPUT_FILE}")
    
except Exception as e:
    print("\n FAILED TO START")
    print(e)

☁️ Configuring for CLOUD/VM Environment...
🚀 Starting Spark Engine...
:: loading settings :: url = jar:file:/home/vanac/.local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vanac/.ivy2/cache
The jars for the packages stored in: /home/vanac/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c1dd641-323a-41a4-9eaa-ef8ede938807;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.36.1 in central
:: resolution report :: resolve 424ms :: artifacts dl 15ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.36.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	------------------------------------------------------------------

 SUCCESS: Spark 3.4.1 is running.
 Target File: gs://vehicles_csv/vehicles.csv


In [3]:
print("--- Step 1: Ingestion ---")
# Load raw data
df_raw = spark.read.csv(INPUT_FILE, header=True, inferSchema=True)
print(f"Raw Count: {df_raw.count()}")

--- Step 1: Ingestion ---


[Stage 2:===============================================>          (9 + 2) / 11]

Raw Count: 441802


In [7]:
from pyspark.sql.functions import col, upper, lower
from pyspark.sql.types import IntegerType, DoubleType

print("--- Step 2: Preprocessing & Standardization ---")

# 1. Standardize Strings (Remove whitespace, uppercase)
df_prep = df_raw \
    .withColumn("manufacturer", upper(col("manufacturer"))) \
    .withColumn("model", upper(col("model"))) \
    .withColumn("title_status", lower(col("title_status"))) \
    .withColumn("vin", upper(col("VIN"))) # Standardize VIN column name

# 2. Handle Nulls (Based on Assumptions)
df_clean = df_prep \
    .dropna(subset=["manufacturer", "model", "year", "price"]) \
    .filter(col("price") > 500) \
    .filter(col("price") < 200000) \
    .fillna({"odometer": -1, "title_status": "clean"}) # Assumption: Missing title = clean

# 3. Type Casting
df_clean = df_clean \
    .withColumn("year", col("year").cast(IntegerType())) \
    .withColumn("odometer", col("odometer").cast(DoubleType()))

print(f"Cleaned Count: {df_clean.count()}")
df_clean.select("manufacturer", "model", "year", "price", "odometer", "title_status").show(5)

--- Step 2: Preprocessing & Standardization ---


Cleaned Count: 364062
+------------+--------------------+----+-----+--------+------------+
|manufacturer|               model|year|price|odometer|title_status|
+------------+--------------------+----+-----+--------+------------+
|         GMC|SIERRA 1500 CREW ...|2014|33590| 57923.0|       clean|
|   CHEVROLET|      SILVERADO 1500|2010|22590| 71229.0|       clean|
|   CHEVROLET| SILVERADO 1500 CREW|2020|39590| 19160.0|       clean|
|      TOYOTA|TUNDRA DOUBLE CAB SR|2017|30990| 41124.0|       clean|
|        FORD|           F-150 XLT|2013|15000|128000.0|       clean|
+------------+--------------------+----+-----+--------+------------+
only showing top 5 rows



In [8]:
print("--- Step 2: Statistical Scoring ---")

# 1. Define the Window
# "Group by Manufacturer, Model, and Year"
windowSpec = Window.partitionBy("manufacturer", "model", "year")

# 2. Calculate Group Statistics (Average & StdDev)
df_stats = df_clean \
    .withColumn("market_avg_price", round(mean("price").over(windowSpec), 2)) \
    .withColumn("market_std_dev", round(stddev("price").over(windowSpec), 2)) \
    .withColumn("group_count", count("id").over(windowSpec))

# 3. Calculate Z-Score
# Formula: (Price - Average) / Standard Deviation
# Logic: How far is this specific car from the 'normal' price?
df_scored = df_stats.withColumn("price_z_score",
    (col("price") - col("market_avg_price")) / when(col("market_std_dev") == 0, 1).otherwise(col("market_std_dev"))
)

# 4. Assign Ratings
df_scored_labeled = df_scored.withColumn("deal_rating",
    when(col("group_count") < 5, "INSUFFICIENT_DATA")  # Skip rare cars
    .when(col("price_z_score") < -1.5, "GREAT DEAL")   # Cheap
    .when(col("price_z_score").between(-1.5, -0.5), "GOOD PRICE")
    .when(col("price_z_score").between(-0.5, 0.5), "FAIR MARKET")
    .when(col("price_z_score") > 1.5, "OVERPRICED")    # Expensive
    .otherwise("SLIGHTLY HIGH")
)

print("Scoring Complete. Preview:")
df_scored_labeled.select("manufacturer", "model", "price", "market_avg_price", "deal_rating").show(10)

--- Step 2: Statistical Scoring ---
Scoring Complete. Preview:


[Stage 11:>                                                         (0 + 1) / 1]

+------------+-----+-----+----------------+-----------+
|manufacturer|model|price|market_avg_price|deal_rating|
+------------+-----+-----+----------------+-----------+
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
|        2015| 2016| 2013|          2013.0|FAIR MARKET|
+------------+-----+-----+----------------+-----------+
only showing top 10 rows



In [10]:
from pyspark.sql.functions import col, substring, when, abs as spark_abs, lit

print("--- Step 3: VIN Forensics & Fraud Detection ---")

# 1. VIN Year Logic (Simplified for 2010-2029 codes)
# Position 10 in VIN indicates year. 
# A=2010, B=2011 ... H=2017, J=2018, K=2019, L=2020, M=2021, N=2022, P=2023
df_fraud_check = df_clean.withColumn("vin_year_code", substring(col("vin"), 10, 1))

# Map code to year (SQL-style Case Statement)
df_fraud_check = df_fraud_check.withColumn("decoded_vin_year",
    when(col("vin_year_code") == "A", 2010)
    .when(col("vin_year_code") == "B", 2011)
    .when(col("vin_year_code") == "C", 2012)
    .when(col("vin_year_code") == "D", 2013)
    .when(col("vin_year_code") == "E", 2014)
    .when(col("vin_year_code") == "F", 2015)
    .when(col("vin_year_code") == "G", 2016)
    .when(col("vin_year_code") == "H", 2017)
    .when(col("vin_year_code") == "J", 2018)
    .when(col("vin_year_code") == "K", 2019)
    .when(col("vin_year_code") == "L", 2020)
    .when(col("vin_year_code") == "M", 2021)
    .when(col("vin_year_code") == "N", 2022)
    .when(col("vin_year_code") == "P", 2023)
    .otherwise(0) # 0 means older than 2010 or not mapped
)

# 2. Fraud Logic
# Flag 1: Year Mismatch (VIN says 2012, Listing says 2018)
# Flag 2: Suspicious Low Mileage (Age > 5 years but odometer < 2000 miles)
current_yr = 2025
df_fraud_check = df_fraud_check \
    .withColumn("is_year_mismatch", 
        when((col("decoded_vin_year") > 0) & (spark_abs(col("decoded_vin_year") - col("year")) > 1), True)
        .otherwise(False)
    ) \
    .withColumn("is_suspicious_mileage", 
        when((col("year") < (current_yr - 5)) & (col("odometer") > 0) & (col("odometer") < 2000), True)
        .otherwise(False)
    )

# Trust Score (0 to 100)
# Start at 100. Subtract 50 for Year Mismatch. Subtract 30 for Suspicious Mileage.
df_fraud_check = df_fraud_check.withColumn("trust_score", 
    lit(100) - 
    when(col("is_year_mismatch"), 50).otherwise(0) - 
    when(col("is_suspicious_mileage"), 30).otherwise(0)
)

print("Fraud Detection Preview (Low Trust Items):")
df_fraud_check.filter(col("trust_score") < 100).select("vin", "year", "decoded_vin_year", "odometer", "trust_score").show(5)

--- Step 3: VIN Forensics & Fraud Detection ---
Fraud Detection Preview (Low Trust Items):
+-----------------+----+----------------+--------+-----------+
|              vin|year|decoded_vin_year|odometer|trust_score|
+-----------------+----+----------------+--------+-----------+
|1FTER4EH3KLA31326|2019|            2019|  1834.0|         70|
|1C4GJXAGXKW566358|2019|            2019|  1423.0|         70|
|             null|1976|               0|   100.0|         70|
|             null|1979|               0|  1234.0|         70|
|1C4AJWAG1HL698869|2017|            2017|   891.0|         70|
+-----------------+----+----------------+--------+-----------+
only showing top 5 rows



In [11]:
print("--- Step 4: Geospatial Arbitrage ---")

# 1. Calculate National Average for every Make/Model/Year
national_window = Window.partitionBy("manufacturer", "model", "year")
df_geo = df_fraud_check.withColumn("national_avg_price", round(mean("price").over(national_window), 2))

# 2. Calculate Regional Arbitrage
# Arbitrage Value = National Average - Local Price
# Positive Value = Cheaper to buy here (Good for flipping)
# Negative Value = More expensive here (Bad for buying)
df_geo = df_geo.withColumn("arbitrage_opportunity", col("national_avg_price") - col("price"))

# 3. Flag High-Value Arbitrage Regions
# If you can save > $2000 by buying here vs national average, it's a "Sourcing Hub"
df_geo = df_geo.withColumn("is_sourcing_hub", when(col("arbitrage_opportunity") > 2000, True).otherwise(False))

print("Arbitrage Preview (Top Opportunities):")
df_geo.filter("is_sourcing_hub").select("region", "manufacturer", "model", "price", "national_avg_price", "arbitrage_opportunity").show(5)

--- Step 4: Geospatial Arbitrage ---
Arbitrage Preview (Top Opportunities):


[Stage 15:>                                                         (0 + 1) / 1]

+--------------------+------------+-------------+-----+------------------+---------------------+
|              region|manufacturer|        model|price|national_avg_price|arbitrage_opportunity|
+--------------------+------------+-------------+-----+------------------+---------------------+
|macon / warner ro...|       ACURA|3.2 CL TYPE S| 6000|            8389.4|   2389.3999999999996|
|             chicago|       ACURA|3.2 CL TYPE S| 5950|            8389.4|   2439.3999999999996|
|     fresno / madera|       ACURA|          ILX| 8000|          10949.53|   2949.5300000000007|
|ft myers / SW flo...|       ACURA|          ILX| 7988|          10949.53|   2961.5300000000007|
|       south florida|       ACURA|          ILX| 7900|          10949.53|   3049.5300000000007|
+--------------------+------------+-------------+-----+------------------+---------------------+
only showing top 5 rows



In [12]:
print("--- Step 5: Adjusted Deal Scoring ---")

windowSpec = Window.partitionBy("manufacturer", "model", "year")

df_scored = df_geo \
    .withColumn("market_std_dev", round(stddev("price").over(windowSpec), 2)) \
    .withColumn("group_count", count("id").over(windowSpec))

# Z-Score Calculation
df_scored = df_scored.withColumn("price_z_score",
    (col("price") - col("national_avg_price")) / when(col("market_std_dev") == 0, 1).otherwise(col("market_std_dev"))
)

# Rating Logic (Integrated with Fraud Check)
# If Trust Score is Low, it cannot be a "Great Deal" (It's a trap!)
df_final = df_scored.withColumn("deal_rating",
    when(col("trust_score") < 70, "RISKY_BUY") # New Category
    .when(col("group_count") < 5, "INSUFFICIENT_DATA")
    .when(col("price_z_score") < -1.5, "GREAT_DEAL")
    .when(col("price_z_score").between(-1.5, -0.5), "GOOD_PRICE")
    .when(col("price_z_score").between(-0.5, 0.5), "FAIR_MARKET")
    .otherwise("OVERPRICED")
)

print("Final Scored Data:")
df_final.select("manufacturer", "model", "price", "trust_score", "arbitrage_opportunity", "deal_rating").show(5)

--- Step 5: Adjusted Deal Scoring ---
Final Scored Data:


[Stage 16:==================================================>     (10 + 1) / 11]

+------------+-----+-----+-----------+---------------------+-----------+
|manufacturer|model|price|trust_score|arbitrage_opportunity|deal_rating|
+------------+-----+-----+-----------+---------------------+-----------+
|        2015| 2016| 2013|        100|                  0.0|FAIR_MARKET|
|        2015| 2016| 2013|        100|                  0.0|FAIR_MARKET|
|        2015| 2016| 2013|        100|                  0.0|FAIR_MARKET|
|        2015| 2016| 2013|        100|                  0.0|FAIR_MARKET|
|        2015| 2016| 2013|        100|                  0.0|FAIR_MARKET|
+------------+-----+-----+-----------+---------------------+-----------+
only showing top 5 rows



In [15]:
from pyspark.sql.functions import current_timestamp

print("--- Step 6: Writing to BigQuery ---")

full_table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}"

try:
    # Select only valuable columns
    output_df = df_final.select(
        "id", "manufacturer", "model", "year", "price", "odometer", "region", "lat", "long",
        "vin", "trust_score", "is_year_mismatch", # Fraud features
        "national_avg_price", "arbitrage_opportunity", "is_sourcing_hub", # Geo features
        "deal_rating", # Final score
        current_timestamp().alias("processed_at")
    )
    
    print(f"🚀 Writing to: {full_table_id}")
    output_df.write \
        .format("bigquery") \
        .option("table", full_table_id) \
        .option("temporaryGcsBucket", GCS_TEMP_BUCKET) \
        .mode("overwrite") \
        .save()
        
    print(" SUCCESS! Market Intelligence Data is in BigQuery.")
    
except Exception as e:
    print(" ERROR writing to BigQuery.")
    print(e)

--- Step 6: Writing to BigQuery ---
🚀 Writing to: de2025-472211.vehicles_data.market_intelligence_system


 SUCCESS! Market Intelligence Data is in BigQuery.
